<a href="https://colab.research.google.com/github/Paul-mwaura/Gender-Based-Discrimination-NLP/blob/main/GBV_Sentiment_Analysis_TextBlob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GBV Sentiment Analysis 

Lets get the sentiment and polarity of the tweets-attitude or emotion of thetext

Short Description

### Resources
* Towards DS: [A brief intro to sentiment Analysis](https://towardsdatascience.com/sentiment-analysis-concept-analysis-and-applications-6c94d6f58c17)
*Monkey learn: [S A](https://monkeylearn.com/blog/sentiment-analysis-machine-learning/)
*Pluralsight: [Texblob](https://www.pluralsight.com/guides/natural-language-processing-extracting-sentiment-from-text-data)

### Import Libraries

In [ ]:
%%capture 
#Import libraries
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

#Text formatting
import nltk
nltk.download('stopwords')

#Installing Texblob
!pip install -U textblob
!python -m textblob.download_corpora

from textblob import TextBlob, Word, Blobber

In [ ]:
text=TextBlob("Damn! I hate Math")
print(text)
text.sentiment

Damn! I hate Math


Sentiment(polarity=-0.8, subjectivity=0.9)

Explaining the output. The polarity is -0.8 which is close to -1. This implies that this is a very negative statement. Polarity lies between -1 to 1 with 1 showing high positivity. 
Subjectivity lies between 0 and 1. Values closer to 1 show that the statement is most probably a public opinion and not a factual piece of information.  Hatred for Maths seems to be a public opinion with no facts

### Load the Dataset

In [ ]:
df = pd.read_csv("GBV_data_clean_v01.csv", error_bad_lines=False)
print(df.shape)
df.head()

(45797, 8)


,Unnamed: 0,username,new_tweets,clean_tweets,tweets_without_stopwords,replies,retweets,likes
0,0,shaybspears,We should talk more about how this dangerous w...,we should talk more about how this dangerous w...,talk dangerous woman stalking britney sending ...,1,14,27
1,1,kaylanicole9991,"And no, getting me drunk wonât work. After w...",and no getting me drunk won t work after what...,getting drunk work happened male best friend j...,1,0,0
2,2,marge78355824,Itâs not just about seeing a random penis...,it s not just about seeing a random penis ...,seeing random penis fear every woman sexual vi...,1,0,1
3,3,explorevenango,A Franklin woman who left multiple messages in...,a franklin woman who left multiple messages in...,franklin woman left multiple messages attempt ...,0,0,0
4,4,politikscommon,I did read the details. Walker was also sue...,i did read the details walker was also sued...,read details walker also sued woman accused wa...,1,0,0


In [ ]:
#drop Nas
df.dropna(inplace=True, axis=0)
df.isna().sum()
tweet_df=df.copy()

Sentiment Analysis
We use the Flair Classifier to get Polarity and Result and add those fields to our dataframe.

In [ ]:
df['clean_tweets'][0]

'we should talk more about how this dangerous woman was stalking britney sending her creepy emails about   a gay demon   and britney reported her to her lawyer but instead of issuing a restraining order against her jamie spears made her brit'

In [ ]:
#Lets check the sentiment of the first 3 tweets
df['clean_tweets'][:2].apply(lambda y: TextBlob(y).sentiment)

0    (-0.04583333333333332, 0.7458333333333333)
1                   (0.18333333333333335, 0.55)
Name: clean_tweets, dtype: object

In [ ]:
#We only extract the polarity and subjectivity of all the clean tweets
# df['polarity'] = df['clean_tweets'][:20].apply(lambda y: TextBlob(y).sentiment[0])
# df['subjectivity']=df['clean_tweets'][:20].apply(lambda y: TextBlob(y).sentiment[1])
polarity=[]
subjectivity=[]
sentiment=[]
for text in df['clean_tweets']:
  text=TextBlob(text)
  sentiment=text.sentiment
  polarity.append(sentiment[0])
  subjectivity.append(sentiment[1])
df['polarity']=polarity
df['subjectivity']=subjectivity


  #print(sentiment[0],sentiment[1])
df.head(10)


,Unnamed: 0,username,new_tweets,clean_tweets,tweets_without_stopwords,replies,retweets,likes,polarity,subjectivity
0,0,shaybspears,We should talk more about how this dangerous w...,we should talk more about how this dangerous w...,talk dangerous woman stalking britney sending ...,1,14,27,-0.045833,0.745833
1,1,kaylanicole9991,"And no, getting me drunk wonât work. After w...",and no getting me drunk won t work after what...,getting drunk work happened male best friend j...,1,0,0,0.183333,0.550000
2,2,marge78355824,Itâs not just about seeing a random penis...,it s not just about seeing a random penis ...,seeing random penis fear every woman sexual vi...,1,0,1,0.062500,0.553333
3,3,explorevenango,A Franklin woman who left multiple messages in...,a franklin woman who left multiple messages in...,franklin woman left multiple messages attempt ...,0,0,0,0.106250,0.220833
4,4,politikscommon,I did read the details. Walker was also sue...,i did read the details walker was also sued...,read details walker also sued woman accused wa...,1,0,0,0.050000,0.350000
5,5,lulabelldesigns,ALERT: Devin Nunes needs your help ASAP. We MU...,alert devin nunes needs your help asap we must...,alert devin nunes needs help asap must defend ...,0,1,3,0.000000,0.000000
6,6,cole_wex,It's amazing that when you want to defend C...,its amazing that when you want to defend ch...,amazing want defend chauncey subject ask defen...,1,0,1,0.127778,0.504167
7,7,blunthonesty4,Last I heard unfortunately it's similar in ...,last i heard unfortunately its similar in a...,last heard unfortunately similar aus woman rap...,1,0,6,-0.125926,0.548148
8,8,seanrosssapp,"Oh, you're talking about the guy who implied...",oh youre talking about the guy who implied t...,oh youre talking guy implied illegitimate chil...,0,0,2,0.150000,0.150000
9,9,menopausemum,Well I donât see why we should pay him a p...,well i don t see why we should pay him a pe...,well see pay penny groping side chicks assbut ...,0,0,0,0.000000,0.000000


In [ ]:
df['polarity'].sort_values(ascending=False)

45356    1.0
45698    1.0
42761    1.0
40852    1.0
40779    1.0
        ... 
17356   -1.0
10431   -1.0
42113   -1.0
45173   -1.0
30740   -1.0
Name: polarity, Length: 45315, dtype: float64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45315 entries, 0 to 45795
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                45315 non-null  int64  
 1   username                  45315 non-null  object 
 2   new_tweets                45315 non-null  object 
 3   clean_tweets              45315 non-null  object 
 4   tweets_without_stopwords  45315 non-null  object 
 5   replies                   45315 non-null  object 
 6   retweets                  45315 non-null  object 
 7   likes                     45315 non-null  object 
 8   polarity                  45315 non-null  float64
 9   subjectivity              45315 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 5.1+ MB


In [ ]:
df['sentiments'] = df.polarity.replace().where(df.polarity > 0, "negative").where(df.polarity != 0, "neutral").where(df.polarity < 0.0000001, "positive")

df.head(20)

,Unnamed: 0,username,new_tweets,clean_tweets,tweets_without_stopwords,replies,retweets,likes,polarity,subjectivity,sentiments
0,0,shaybspears,We should talk more about how this dangerous w...,we should talk more about how this dangerous w...,talk dangerous woman stalking britney sending ...,1,14,27,-0.045833,0.745833,negative
1,1,kaylanicole9991,"And no, getting me drunk wonât work. After w...",and no getting me drunk won t work after what...,getting drunk work happened male best friend j...,1,0,0,0.183333,0.550000,positive
2,2,marge78355824,Itâs not just about seeing a random penis...,it s not just about seeing a random penis ...,seeing random penis fear every woman sexual vi...,1,0,1,0.062500,0.553333,positive
3,3,explorevenango,A Franklin woman who left multiple messages in...,a franklin woman who left multiple messages in...,franklin woman left multiple messages attempt ...,0,0,0,0.106250,0.220833,positive
4,4,politikscommon,I did read the details. Walker was also sue...,i did read the details walker was also sued...,read details walker also sued woman accused wa...,1,0,0,0.050000,0.350000,positive
5,5,lulabelldesigns,ALERT: Devin Nunes needs your help ASAP. We MU...,alert devin nunes needs your help asap we must...,alert devin nunes needs help asap must defend ...,0,1,3,0.000000,0.000000,neutral
6,6,cole_wex,It's amazing that when you want to defend C...,its amazing that when you want to defend ch...,amazing want defend chauncey subject ask defen...,1,0,1,0.127778,0.504167,positive
7,7,blunthonesty4,Last I heard unfortunately it's similar in ...,last i heard unfortunately its similar in a...,last heard unfortunately similar aus woman rap...,1,0,6,-0.125926,0.548148,negative
8,8,seanrosssapp,"Oh, you're talking about the guy who implied...",oh youre talking about the guy who implied t...,oh youre talking guy implied illegitimate chil...,0,0,2,0.150000,0.150000,positive
9,9,menopausemum,Well I donât see why we should pay him a p...,well i don t see why we should pay him a pe...,well see pay penny groping side chicks assbut ...,0,0,0,0.000000,0.000000,neutral


In [ ]:
# sentiment={}
# for val in df['polarity']:
#   if float(val) <0:
#     sentiment="Negative"
#   elif int(val) ==0:
#     sentiment="Neutral"
#   else:
#     sentiment="Positive"
#   sentiment.join(sentiment)
# df['sentiment']=sentiment

In [ ]:
df.sample(50)